In [3]:
import numpy as np

### init random seeds
rd_seed = 13246
np.random.seed(rd_seed)

from sklearn.pipeline import Pipeline
from pinard import preprocessor as pp

### Declare preprocessing pipeline components
preprocessing = [   ('id', pp.IdentityTransformer()),
                    ('savgol', pp.SavitzkyGolay()),
                    ('derivate', pp.Derivate()), 
                    ('gaussian1', pp.Gaussian(order = 1, sigma = 2)),
                    ('gaussian2', pp.Gaussian(order = 2, sigma = 1)),
                    ('haar', pp.Wavelet('haar')),
                    ('savgol*savgol', Pipeline([('_sg1',pp.SavitzkyGolay()),('_sg2',pp.SavitzkyGolay())])),
                    ('gaussian1*savgol', Pipeline([('_g1',pp.Gaussian(order = 1, sigma = 2)),('_sg3',pp.SavitzkyGolay())])),
                    ('gaussian2*savgol', Pipeline([('_g2',pp.Gaussian(order = 1, sigma = 2)),('_sg4',pp.SavitzkyGolay())])),
                    ('haar*savgol', Pipeline([('_haar2',pp.Wavelet('haar')),('_sg5',pp.SavitzkyGolay())]))
                ]

In [8]:
from pinard import nirs_set as n_set

### Load data
n = n_set.NIRS_Set('data')
X, y = n.load('Xcal.csv', 'Ycal.csv', x_hdr=0, y_hdr=0, y_cols=0)
print(X[0:10], y[0:10])

[[1.0522316  1.0564474  1.0320108  ... 0.5621798  0.56231886 0.5622432 ]
 [0.91029036 0.9535987  0.9861997  ... 0.5564785  0.5562965  0.55541193]
 [0.7547634  0.77333385 0.783736   ... 0.46846136 0.46816158 0.4678402 ]
 ...
 [0.6774115  0.6749243  0.6887683  ... 0.45818323 0.45851737 0.4584935 ]
 [0.7275998  0.7463403  0.7508709  ... 0.44724375 0.44707114 0.44744077]
 [0.7024517  0.70507336 0.71183866 ... 0.45939925 0.45958665 0.4589455 ]] [50.04759598 43.89248657 51.19988632 46.24049759 53.12453461 43.63124466
 33.20572281 48.34818268 51.57919693 42.81207275]


In [10]:
## Simple xgboost pipeline
from xgboost import XGBRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

xgb =  XGBRegressor()
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureUnion(preprocessing)), 
    ('XGB', xgb)
])

estimator = TransformedTargetRegressor(regressor = pipeline, transformer = MinMaxScaler())

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = rd_seed)
estimator.fit(X_train, y_train)
Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))


MAE 1.0816270596217166
MSE 2.4316124872821887
MAPE 0.024215514567072056
R² 0.7906370594445927


In [19]:
from sklearn.cross_decomposition import PLSRegression

## Simple PLS pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureUnion(preprocessing)), 
    ('pls', PLSRegression(n_components=10))
])

estimator = TransformedTargetRegressor(regressor = pipeline, transformer = MinMaxScaler())

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = rd_seed)
estimator.fit(X_train, y_train)
Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))


MAE 0.9239768627451542
MSE 1.6106460074820232
MAPE 0.020502684600532527
R² 0.8613226465631612


In [13]:
## KERAS Model
##TODO > remove keraswrapper for sciKeras wrapper

from pinard.nirs_pipelines import FeatureAugmentation

import tensorflow
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, SpatialDropout1D,BatchNormalization,Flatten, Dropout

tensorflow.random.set_seed(rd_seed)

def keras_model(optimizer = 'adam'):
    model = Sequential()
    model.add(SpatialDropout1D(0.08))
    model.add(Conv1D (filters=8, kernel_size=15, strides=5, activation='selu'))
    model.add(Dropout(0.2))
    model.add(Conv1D (filters=64, kernel_size=21, strides=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D (filters=32, kernel_size=5, strides=3, activation='elu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss = 'mean_squared_error', optimizer = optimizer, metrics = ['mae', 'mse'])
    return model

pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureAugmentation(preprocessing)), 
    ('KerasNN',  KerasRegressor(build_fn = keras_model, epochs=2000, batch_size=1000, verbose = 0))
])

estimator = TransformedTargetRegressor(regressor = pipeline, transformer = MinMaxScaler())

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = rd_seed)
estimator.fit(X_train, y_train)
Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))

c:\Workspace\ML\PyNIRS_env\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


TRANSFORM
ahahah (91, 2151, 10)
MAE 1.4922053993064257
MSE 5.164645265859251
MAPE 0.03387224602967861
R² 0.5553216947843496


In [23]:
##Explainer

import shap

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
# import seaborn as sns # for correlation heatmap

# perm_importance = permutation_importance(xgb, X_test, y_test)
# sorted_idx = perm_importance.importances_mean.argsort()
# plt.barh(np.arrange(0,len(X_test[0],1))[sorted_idx], perm_importance.importances_mean[sorted_idx])
# plt.xlabel("Permutation Importance")
explainer = shap.Explainer(estimator.predict, X_train)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")


Exception: The passed model is not callable and cannot be analyzed directly with the given masker! Model: TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler', MinMaxScaler()),
                                                     ('preprocessing',
                                                      FeatureUnion(transformer_list=[('id',
                                                                                      FunctionTransformer()),
                                                                                     ('savgol',
                                                                                      SavitzkyGolay()),
                                                                                     ('derivate',
                                                                                      Derivate()),
                                                                                     ('gaussian1',
                                            